In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, subprocess
%load_ext watermark

In [ ]:
%watermark

In [ ]:
#parameters
workingdir = r"/Volumes/GoogleDrive/My Drive/tesi_magistrale/tesi/gtex/hsbm/random_genes"
#workingdir=r"/Volumes/GoogleDrive/My Drive/tesi_magistrale/tesi/results/hsbm/miRNA"
execdir = r"/Users/filippo/Developer/tesi"
#dockerurl = "cloudadm@193.144.35.220:/home/cloudadm/master_thesis/hsbm"
dockerurl = "filippo.valle@c3s.unito.it:/archive/home/filippo.valle/hsbm-occam/tcgamirna"

In [ ]:
os.chdir(workingdir)

# Get the data
[Table_creation](Table_creation.ipynb) and [GTex_table_creation](GTex_table_creation) do this for you

# Get annotations

In [ ]:
df_file=pd.read_csv("files.dat", index_col=[0])
df_file.head()

In [ ]:
df_file.groupby(by='primary_site').count()

In [ ]:
pc=pd.read_csv("%s/genes.txt"%execdir, index_col=[0])
pc=pc[pc['type_of_gene']=='protein-coding']
pc.head()

# Run statistics
```python
import tacos
tacos.statistics()
tacos.shuffleLabels()
```

CHECK **BioParameters.h** before compiling

In [ ]:
os.chdir(workingdir)
os.system("cp %s/tacos/master/python/addo.py ."%execdir)
os.system("cp %s/tacos/master/data_mining ."%execdir)
os.system("chmod +x data_mining")
os.system("mv mainTable_all.csv mainTable.csv")
os.system("./data_mining 0")
os.system("./data_mining 2")
os.system("python addo.py")

In [ ]:
os.system("./data_mining 8")#shuffle files

# Analyse table and select genes

[Table_Analyser.ipynb](Table_Analyzer.ipynb)

In [ ]:
os.chdir(execdir)
from tableanalyser import scalinglawsandoverexpressed

In [ ]:
scalinglawsandoverexpressed(workingdir)

In [ ]:
os.chdir(workingdir)
os.system("mv mainTable.csv mainTable_all.csv")
os.system("mv mainTable_over.csv mainTable.csv")

# Make bipartite network
```bash
python addo.py
```
```python
import tacos
tacos.makegraph()
```

In [ ]:
os.chdir(workingdir)
os.system("./data_mining 0")
os.system("./data_mining 2")
os.system("python addo.py")
os.system("./data_mining 7")

In [ ]:
os.system("scp graph.xml.gz %s/."%dockerurl)

[hSBM.ipynb]()

# Run stochastic block model
```
docker run -it -d -v $PWD:/home/filippo fvalle01/hsbm
```

download the data

In [ ]:
os.chdir(workingdir)
os.system("mkdir -p topsbm")
os.system("scp %s/* topsbm/."%dockerurl)

# Run cluster analysis

In [ ]:
os.chdir(execdir)
from hsbmpy import clusteranalysis, get_scores, add_score_lines, getclustersizesarray
#geneontology

In [ ]:
os.chdir(execdir)
import hsbmpy, importlib
importlib.reload(hsbmpy)
from hsbmpy import clusteranalysis, get_scores, add_score_lines, getclustersizesarray
os.chdir(workingdir)

In [ ]:
labels = df_file.columns
clusteranalysis(workingdir, labels, l=3)

In [ ]:
scores = get_scores(workingdir,labels, l=3, verbose=False)
fig=plt.figure(figsize=(12,8))
ax = fig.subplots(1)
add_score_lines(ax,scores,labels=[k for k in scores.keys()],xl=getclustersizesarray(workingdir,l=3))
plt.show()
fig.savefig("%s/metric.pdf"%(workingdir))

In [ ]:
geneontology.topicanalysis()

# Clean

In [ ]:
os.chdir(workingdir)
os.system("rm -rf data_mining addo.py")